In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.cluster import KMeans


# Our functions import

In [2]:
from generate_population_scripts import (generate_population,
                                         evaluate_distance_fitness, 
                                         evaluate_fitness_customers,
                                         get_evalution_count, 
                                         set_evaluation_count)
from combination import cut_and_crossfill, mutation_inverse, mutation_scramble
from selection import roulette_wheel, binary_tournament

In [6]:
def algorithm_run(pop_count, dataset, depot_location_dict, depot_symbol, vehicle_count, selection_method, FITNESS_FUNCTION, MUTATION_METHOD, RECOMBINATION_METHOD, max_capacity=None, max_distance=None, p_m=0.1, p_c =0.9 ,max_evaluations = 5000):
    """
    depot_symbol is just one string

    one constraint should be always given as input, the maximum capacity or maximum distance
    """
    ## reset the evaluation counts
    set_evaluation_count(0)

    population, fitness_pop = generate_population(max_capacity=max_capacity,
                                                     max_distance=max_distance,
                                                     DEPOT_LOCATION= depot_location_dict[depot_symbol],
                                                     dataset=dataset, 
                                                     depot_symbol= depot_symbol, 
                                                     pop_count= pop_count,
                                                     vehicle_count= vehicle_count, )
    generation_count = 0

    best_chromosome = None
    best_chromsome_fitness = None

    while get_evalution_count() < max_evaluations:
        print(f'Generation Count: {generation_count}, Evaluations number: {get_evalution_count()}')
        ## create pair of the parents
        parent_pairs = []
        for _ in range(pop_count):
            pair = selection_method(population, fitness_pop)
            parent_pairs.append(pair)

        
        offsprings = []
        fitness_offsprings = []
        for parents in parent_pairs:
            recombination_p = np.random.random()

            ## the offspring for this iteration
            iteration_offspring = []
            
            ######## Recombination ########
            if recombination_p < p_c:
                offspring1, offspring2 =  RECOMBINATION_METHOD(parents[0], parents[1], dataset, max_capacity, max_distance, depot_location_dict[depot_symbol], [depot_symbol])

                iteration_offspring = [offspring1, offspring2]

            
            ######## Mutation ########
            mutation_p = np.random.random()

            ## if cross over has happend
            if len(iteration_offspring) != 0 and mutation_p < p_m:
                offspring1 = MUTATION_METHOD(iteration_offspring[0], max_capacity, dataset, depot_location_dict[depot_symbol], max_distance, [depot_symbol])
                offspring2 = MUTATION_METHOD(iteration_offspring[1], max_capacity, dataset, depot_location_dict[depot_symbol], max_distance, [depot_symbol])

                iteration_offspring = [offspring1, offspring2]
            ## if cross over had not happened
            elif mutation_p < p_m:
                offspring1 = MUTATION_METHOD(parents[0], max_capacity, dataset, depot_location_dict[depot_symbol], max_distance, [depot_symbol])
                offspring2 = MUTATION_METHOD(parents[1], max_capacity, dataset, depot_location_dict[depot_symbol], max_distance, [depot_symbol])

                iteration_offspring = [offspring1, offspring2]
            ## if both cross over and mutation has not happened!
            else:
                iteration_offspring = [parents[0], parents[1]]
            
            ## finally append the genarated offsprings to offspring array 
            offsprings.append(iteration_offspring[0])
            offsprings.append(iteration_offspring[1])

            fitness_offsprings.append(FITNESS_FUNCTION(iteration_offspring[0], depot_location_dict[depot_symbol], dataset))
            fitness_offsprings.append(FITNESS_FUNCTION(iteration_offspring[1], depot_location_dict[depot_symbol], dataset))

        ## replace the old population with the new ones

        ## the whole generation: parents + offsprings
        generation_population = population.copy()
        generation_population.extend(offsprings)

        ## whole generation fitness: parents fitness + offsprings fitness
        generation_fitness = fitness_pop.copy()
        generation_fitness.extend(fitness_offsprings)

        ## the sorted generation
        generation_population_sorted = np.array(generation_population)[np.argsort(generation_fitness)]
        generation_fitness_sorted = np.sort(generation_fitness)

        ## Step 10
        ## extract the best of the new generation

        best_of_generation_population = generation_population_sorted[:pop_count]
        best_of_generation_fitness = generation_fitness_sorted[:pop_count]

        best_chromosome = generation_population_sorted[0]
        best_chromsome_fitness = generation_fitness_sorted[0]

        ## save them into the original population arrays
        population = best_of_generation_population.tolist()
        fitness_pop = best_of_generation_fitness.tolist()

        ## increase the generation value
        generation_count += 1
    
    return best_chromosome, best_chromsome_fitness

# PROBLEM No. 1
The goal is to minimize the distance gone, of the vehicles with the parameters below
- There is 1 depot in the location `(-14, 9)`
- There are `6` vehicles
- Maximum Capacity is `70`
- The locations of the customers and their demands are in `P1.txt` file
- The distance each vehicle can go is not limited.

In [7]:
p1_data = pd.read_csv('data/P1.txt', delimiter=' ')
DEPOT_LOCATION = (-14, 9)

pop_arr = generate_population(70, DEPOT_LOCATION, p1_data)

In [8]:
problem1_best_chromsome, problem1_best_chromsome_fitness = algorithm_run(pop_count = 50, 
                dataset=p1_data,
                depot_location_dict= {'(1)': DEPOT_LOCATION},
                depot_symbol='(1)',
                max_capacity=70, 
                vehicle_count=6, 
                max_distance=None,
                selection_method=binary_tournament,
                FITNESS_FUNCTION=evaluate_distance_fitness,
                MUTATION_METHOD=mutation_scramble,
                RECOMBINATION_METHOD=cut_and_crossfil,
                max_evaluations=5000)

problem1_best_chromsome, problem1_best_chromsome_fitness

NameError: name 'cut_and_crossfil' is not defined

# Problem No. 2
The goal is to maximize the customers recieved service with parametes below
- There is 1 depot in the location `(0, 13)`
- There are `4` vehicles
- Maximum distace each vehicle can go is `200` Km
- The locations of the customers and their demands are in `P2.txt` file
- The capacity is not limited for vehicles

In [7]:
p2_data = pd.read_csv('data/P2.txt', delimiter=' ')
DEPOT_LOCATION = (0, 13)

In [8]:
problem2_best_chromsome, problem2_best_chromsome_fitness = algorithm_run(pop_count = 50, 
                dataset=p2_data,
                depot_location_dict= {'(1)': DEPOT_LOCATION},
                depot_symbol='(1)',
                max_capacity=None, 
                max_distance=200,
                vehicle_count=4, 
                selection_method=binary_tournament,
                FITNESS_FUNCTION=evaluate_fitness_customers,
                MUTATION_METHOD=mutation_scramble,
                RECOMBINATION_METHOD=cut_and_crossfill,
                max_evaluations=5000)

problem2_best_chromsome, problem2_best_chromsome_fitness

Generation Count: 0, Evaluations number: 50


('(1)160175106(1)107(1)169(1)148138(1)195(1)177(1)103157168(1)121156(1)|(1)180142129189(1)|(1)109(1)166139185(1)171192(1)165(1)146114(1)183124193(1)173133(1)104178(1)152101172119194(1)153161(1)|(1)155108200(1)117174110125(1)144123(1)162(1)113131(1)154184(1)181134(1)190(1)141115(1)116105112(1)122188(1)150120(1)118(1)197137(1)164(1)127176(1)102182191(1)128167(1)',
 0.01282051282051282)

In [10]:
chromsome = '(1)187168115183178105186(1)101130125147132162(1)138171170182140185(1)150196106155188(1)158116107111139133(1)|(1)108179151135(1)184134103112145163129(1)128192148102173(1)200152114144113(1)169143166120(1)|(1)156175191193(1)161165181(1)157124146154141(1)174136117123(1)|(1)167160176189(1)142121198180172126(1)153122159197(1)164127131194104(1)|(1)195137119110118(1)109199177190(1)|(1)149(1)'
chromsome[-6:]

'149(1)'

In [7]:
# p2_data[abs(p2_data.x - DEPOT_LOCATION[0]) + abs(p2_data.y - DEPOT_LOCATION[0]) > 100]
1 / 0.012195121951219513

82.0